In [9]:
import tensorflow as tf

training_dir = 'code/imgs/face_training'
validation_dir = 'code/imgs/face_validation'

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    training_dir,
    # Inception V3 입력 크기 150*150
    target_size=(150, 150),
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    # Inception V3 입력 크기 150*150
    target_size=(150, 150),
    class_mode='categorical')


Found 316 images belonging to 4 classes.
Found 76 images belonging to 4 classes.


In [10]:
# import urllib


# weights_url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
# weights_file = 'models/inception_v3.h5'

# # weights_url에서 파일을 가져와 weights_file이라는 이름으로 저장
# urllib.request.urlretrieve(weights_url, weights_file)

weights_file = 'code/models/inception_v3.h5'

# Inception V3 신경망 모델 생성
pre_trained_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape=(150, 150, 3),
    include_top=False,
    weights=None)

# 생성된 모델에 가져온 가중치 부여
pre_trained_model.load_weights(weights_file)

In [ ]:
# 모델 정보 출력
pre_trained_model.summary()

In [14]:
# 가져온 신경망의 parameter가 훈련되지 않도록 동결
for layer in pre_trained_model.layers:
    layer.trainable = False

# mixed7 층의 마지막 출력을 가리키는 변수 생성
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

# 출력 펼치기
x = tf.keras.layers.Flatten()(last_output)
# Dense 층 추가
x = tf.keras.layers.Dense(1024, activation='relu')(x)
# 다중 분류를 위해 소프트맥스 함수를 사용한 출력층 생성
x = tf.keras.layers.Dense(4, activation='softmax')(x)

# 모델 생성
model = tf.keras.Model(pre_trained_model.input, x)

# 훈련 방법 설정
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['acc'])

# 훈련 시작
model.fit(train_generator, epochs=40, validation_data=validation_generator)

Epoch 1/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 403ms/step - acc: 0.2916 - loss: 2.7507 - val_acc: 0.4605 - val_loss: 1.3956
Epoch 2/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 265ms/step - acc: 0.4097 - loss: 1.2557 - val_acc: 0.5263 - val_loss: 1.0201
Epoch 3/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 274ms/step - acc: 0.4283 - loss: 1.2218 - val_acc: 0.6842 - val_loss: 0.8702
Epoch 4/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 265ms/step - acc: 0.6104 - loss: 0.8754 - val_acc: 0.6316 - val_loss: 0.9670
Epoch 5/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 271ms/step - acc: 0.6447 - loss: 0.9188 - val_acc: 0.4868 - val_loss: 1.1729
Epoch 6/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 288ms/step - acc: 0.6463 - loss: 0.8438 - val_acc: 0.6579 - val_loss: 0.8203
Epoch 7/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 266ms/step - acc: 0.6049 - loss: 0.9231 - val_acc: 0.6579 - val_loss: 0.7886
Epoch 8/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 266ms/step - acc: 0.6862 - loss: 0.8016 - val_acc: 0.6447 - val_loss: 0.7368
Epoch 9/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 266ms/step - ac

In [81]:
import cv2
import numpy as np

# 이미지 로드
image = cv2.imread('code/imgs/predict/GuDongBin_faces/bin.jpg')

# # 이미지 크기 조정 (150, 150)으로 조정
# image = cv2.resize(image, (150, 150))

# 모델이 기대하는 형태로 차원 추가 (batch 차원 추가)
input_image = np.expand_dims(image, axis=0)

# 이미지를 0과 1사이의 값으로 조정 (정규화)
input_image = input_image/255.0

# 입력 이미지에 대한 예측
model.predict(input_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


array([[3.1007734e-05, 1.3918097e-02, 9.2667090e-03, 9.7678417e-01]],
      dtype=float32)

In [82]:
pred = model.predict(input_image)

# 학습시켰던 연예인 이름 레이블
class_labels = ['이승기', '남주혁', '박보영', '서강준']

# 가장 큰 원소의 인덱스를 반환
predict_class_index = np.argmax(pred)

# 가장 높은 확률을 가진 클래스의 레이블명 반환
predict_label = class_labels[predict_class_index]

print(f'당신이 {predict_label}일 확률은 {round(pred[0][2] * 100, 2)}% 입니다!')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
당신이 서강준일 확률은 0.93% 입니다!
